<i>Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.</i>

# SAR Single Node on MovieLens (Python, CPU)

In this example, we will walkthrough each step of the Smart Adaptive Recommendations (SAR) algorithm with a Python single-node implementation.

SAR is a fast, scalable, adaptive algorithm for personalized recommendations based on user transaction history and item descriptions. It is powered by understanding the similarity between items, and recommending similar items to ones a user has an existing affinity for.

## 1 SAR algorithm

In the next figure a high-level architecture of SAR is showed.
<img src="https://recodatasets.blob.core.windows.net/images/sar_schema.svg?sanitize=true">

### 1.1 Compute item co-occurrence and item similarity

Central to how SAR defines similarity is an item-to-item co-occurrence matrix. Co-occurrence is defined as the number of times two items appear together for a given user. We can represent the co-occurrence of all items as a $m\times m$ matrix $C$, where $c_{i,j}$ is the number of times item $i$ occurred with item $j$.

The co-occurence matric $C$ has the following properties:

It is symmetric, so $c_{i,j} = c_{j,i}$
It is nonnegative: $c_{i,j} \geq 0$
The occurrences are at least as large as the co-occurrences. I.e, the largest element for each row (and column) is on the main diagonal: $\forall(i,j) C_{i,i},C_{j,j} \geq C_{i,j}$.
Once we have a co-occurrence matrix, an item similarity matrix $S$ can be obtained by rescaling the co-occurrences according to a given metric. Options for the metric include Jaccard, lift, and counts (meaning no rescaling).

The rescaling formula for Jaccard is $s_{ij}=c_{ij} / (c_{ii}+c_{jj}-c_{ij})$

and that for lift is $s_{ij}=c_{ij} / (c_{ii} \times c_{jj})$

where $c_{ii}$ and $c_{jj}$ are the $i$th and $j$th diagonal elements of $C$. In general, using counts as a similarity metric favours predictability, meaning that the most popular items will be recommended most of the time. Lift by contrast favours discoverability/serendipity: an item that is less popular overall but highly favoured by a small subset of users is more likely to be recommended. Jaccard is a compromise between the two.


### 1.2 Compute user affinity scores

The affinity matrix in SAR captures the strength of the relationship between each individual user and each item. The event types and weights are used in computing this matrix: different event types (such as “rate” vs “view”) should be allowed to have an impact on a user’s affinity for an item. Similarly, the time of a transaction should have an impact; an event that takes place in the distant past can be thought of as being less important in determining the affinity.

Combining these effects gives us an expression for user-item affinity:

$$a_{ij}=\sum_k (w_k \text{exp}[-\text{log}_2(\frac{t_0-t_k}{T})] $$
where the affinity for user $i$ and item $j$ is the sum of all events involving user $i$ and item $j$, and $w_k$ is the weight of event $k$. The presence of the  $\text{log}_{2}$ factor means that the parameter $T$ in the exponential decay term can be treated as a half-life: events this far before the reference date $t_0$ will be given half the weight as those taking place at $t_0$.

Repeating this computation for all $n$ users and $m$ items results in an $n\times m$ matrix $A$. Simplifications of the above expression can be obtained by setting all the weights equal to 1 (effectively ignoring event types), or by setting the half-life parameter $T$ to infinity (ignoring transaction times).

### 1.3 Remove seen item

Optionally we remove items which have already been seen in the training set, i.e. don't recommend items which have been previously bought by the user again.

### 1.4 Top-k item calculation

The personalized recommendations for a set of users can then be obtained by multiplying the affinity matrix ($A$) by the similarity matrix ($S$). The result is a recommendation score matrix, with one row per user / item pair; higher scores correspond to more strongly recommended items.

It is worth noting that the complexity of recommending operation depends on the data size. SAR algorithm itself has $O(n^3)$ complexity. Therefore the single-node implementation is not supposed to handle large dataset in a scalable manner. Whenever one uses the algorithm, it is recommended to run with sufficiently large memory. 

## 2 SAR single-node implementation

The SAR implementation illustrated in this notebook was developed in Python, primarily with Python packages like `numpy`, `pandas`, and `scipy` which are commonly used in most of the data analytics / machine learning tasks. Details of the implementation can be found in [Recommenders/reco_utils/recommender/sar/sar_singlenode.py](../../reco_utils/recommender/sar/sar_singlenode.py).

## 3 SAR single-node based movie recommender

In [8]:
# set the environment path to find Recommenders
import sys
sys.path.append("../../")

import os
import itertools
import numpy as np
import pandas as pd
import papermill as pm

from reco_utils.recommender.sar.sar_singlenode import SARSingleNode
from reco_utils.dataset import movielens
from reco_utils.dataset.python_splitters import python_random_split
from reco_utils.evaluation.python_evaluation import map_at_k, ndcg_at_k, precision_at_k, recall_at_k

print("System version: {}".format(sys.version))
print("Pandas version: {}".format(pd.__version__))

System version: 3.6.0 | packaged by conda-forge | (default, Feb  9 2017, 14:36:55) 
[GCC 4.8.2 20140120 (Red Hat 4.8.2-15)]
Pandas version: 0.23.4


In [9]:
# top k items to recommend
TOP_K = 10

# Select Movielens data size: 100k, 1m, 10m, or 20m
MOVIELENS_DATA_SIZE = '100k'

### 3.1 Load Data

SAR is intended to be used on interactions with the following schema:
`<User ID>, <Item ID>, <Time>`. 

Each row represents a single interaction between a user and an item. These interactions might be different types of events on an e-commerce website, such as a user clicking to view an item, adding it to a shopping basket, following a recommendation link, and so on. 

The MovieLens dataset is well formatted interactions of Users providing Ratings to Movies (movie ratings are used as the event weight) - we will use it for the rest of the example.

In [10]:
data = movielens.load_pandas_df(
    size=MOVIELENS_DATA_SIZE,
    header=['UserId','MovieId','Rating','Timestamp']
)

# Convert the float precision to 32-bit in order to reduce memory consumption 
data.loc[:, 'Rating'] = data['Rating'].astype(np.float32)

data.head()

,UserId,MovieId,Rating,Timestamp
0,196,242,3.0,881250949
1,186,302,3.0,891717742
2,22,377,1.0,878887116
3,244,51,2.0,880606923
4,166,346,1.0,886397596


### 3.2 Split the data using the python random splitter provided in utilities:

We utilize the provided `python_random_split` function to split into `train` and `test` datasets randomly at a 75/25 ratio.

In [11]:
train, test = python_random_split(data, 0.75)

In [12]:
header = {
        "col_user": "UserId",
        "col_item": "MovieId",
        "col_rating": "Rating",
        "col_timestamp": "Timestamp",
    }

In this case, for the illustration purpose, the following parameter values are used:

|Parameter|Value|Description|
|---------|---------|-------------|
|`similarity_type`|`jaccard`|Method used to calculate item similarity.|
|`time_decay_coefficient`|30|Period in days (term of $T$ shown in the formula of Section 2.2.2)|
|`time_now`|`None`|Time decay reference.|
|`timedecay_formula`|`True`|Whether time decay formula is used.|

In [13]:
model = SARSingleNode(
    remove_seen=True, similarity_type="jaccard", 
    time_decay_coefficient=30, time_now=None, timedecay_formula=True, **header
)

In [14]:
unique_users = data["UserId"].unique()
unique_items = data["MovieId"].unique()

We will hash users and items to smaller continuous space.
This is an ordered set - it's discrete, but contiguous.
This helps keep the matrices we keep in memory as small as possible.

In [15]:
enumerate_items_1, enumerate_items_2 = itertools.tee(enumerate(unique_items))
enumerate_users_1, enumerate_users_2 = itertools.tee(enumerate(unique_users))
item_map_dict = {x: i for i, x in enumerate_items_1}
user_map_dict = {x: i for i, x in enumerate_users_1}

The reverse of the dictionary above - array index to actual ID


In [16]:
index2user = dict(enumerate_users_2)
index2item = dict(enumerate_items_2)

We need to index the train and test sets for SAR matrix operations to work

In [17]:
model.set_index(unique_users, unique_items, user_map_dict, item_map_dict, index2user, index2item)

In [18]:
model.fit(train)

Calculating time-decayed affinities...
../../reco_utils/recommender/sar/sar_singlenode.py:219: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df["exponential"] = expo_fun(df[self.col_timestamp].values)
../../reco_utils/recommender/sar/sar_singlenode.py:221: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df["rating_exponential"] = df[self.col_rating] * df["exponential"]
Creating index columns...
../../reco_utils/recommender/sar/sar_singlenode.py:283: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  s

In [19]:
top_k = model.recommend_k_items(test)

Converting to dense matrix...
../../reco_utils/recommender/sar/sar_singlenode.py:422: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  test[self._col_hashed_users] = test[self.col_user].map(self.user_map_dict)
Removing seen items...
Getting top K...
Select users from the test set
Creating output dataframe...
Formatting output


In [20]:
# TODO: remove this call when the model returns same type as input
top_k['UserId'] = pd.to_numeric(top_k['UserId'])
top_k['MovieId'] = pd.to_numeric(top_k['MovieId'])

The final output from the `recommend_k_items` method generates recommendation scores for each user-item pair, which are shown as follows.

In [21]:
display(top_k.head())

,UserId,MovieId,prediction
0,600,69,12.984131
1,600,423,12.991756
2,600,183,13.106912
3,600,89,13.163791
4,600,144,13.489795


### 3.3 Evaluate the results

It should be known that the recommendation scores generated by multiplying the item similarity matrix $S$ and the user affinity matrix $A$ **DOES NOT** have the same scale with the original explicit ratings in the movielens dataset. That is to say, SAR algorithm is meant for the task of *recommending relevent items to users* rather than *predicting explicit ratings for user-item pairs*. 

To this end, ranking metrics like precision@k, recall@k, etc., are more applicable to evaluate SAR algorithm. The following illustrates how to evaluate SAR model by using the evaluation functions provided in the `reco_utils`.

In [22]:
eval_map = map_at_k(test, top_k, col_user="UserId", col_item="MovieId", 
                    col_rating="Rating", col_prediction="prediction", 
                    relevancy_method="top_k", k=TOP_K)

eval_ndcg = ndcg_at_k(test, top_k, col_user="UserId", col_item="MovieId", 
                      col_rating="Rating", col_prediction="prediction", 
                      relevancy_method="top_k", k=TOP_K)

eval_precision = precision_at_k(test, top_k, col_user="UserId", col_item="MovieId", 
                                col_rating="Rating", col_prediction="prediction", 
                                relevancy_method="top_k", k=TOP_K)

eval_recall = recall_at_k(test, top_k, col_user="UserId", col_item="MovieId", 
                          col_rating="Rating", col_prediction="prediction", 
                          relevancy_method="top_k", k=TOP_K)

In [23]:
print("Model:\t" + model.model_str,
      "Top K:\t%d" % TOP_K,
      "MAP:\t%f" % eval_map,
      "NDCG:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall, sep='\n')

Model:	sar_ref
Top K:	10
MAP:	0.105815
NDCG:	0.373197
Precision@K:	0.326617
Recall@K:	0.175957


## References
Note SAR is a combinational algorithm that implements different industry heuristics. The followings are references that may be helpful in understanding the SAR logic and implementation. 

1. Badrul Sarwar, *et al*, "Item-based collaborative filtering recommendation algorithms", WWW, 2001.
2. Scipy (sparse matrix), url: https://docs.scipy.org/doc/scipy/reference/sparse.html
3. Asela Gunawardana and Guy Shani, "A survey of accuracy evaluation metrics of recommendation tasks", The Journal of Machine Learning Research, vol. 10, pp 2935-2962, 2009.	